## Libraries ##

In [ ]:
import re
import math
import string
import numpy as np
import pandas as pd
from scipy import interp

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import KernelPCA, SparsePCA
from sklearn.decomposition import PCA

import xgboost
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import Perceptron
# from imblearn.over_sampling import SMOTE

from matplotlib import pyplot
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

## Function for computing scores and graphs ##

In [ ]:
def scores(classifier, X_train, X_test, y_train, y_test):
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    predictions_proba = classifier.predict_proba(X_test)
    predictions_proba = predictions_proba[:, 1]
    
    fp, tp, threshold = roc_curve(y_test, predictions_proba)
    precision_position, recall_position, _ = precision_recall_curve(y_test, predictions_proba)
    
    accuracy  = round(accuracy_score(y_test, predictions) * 100, 4)
    auc       = round(roc_auc_score(y_test, predictions_proba) * 100, 4)
    aupr      = round(average_precision_score(y_test, predictions_proba) * 100, 4)
    precision = round(precision_score(y_test, predictions, average='binary') * 100, 4)
    recall    = round(recall_score(y_test, predictions, average='binary') * 100, 4)
    f1        = round(f1_score(y_test, predictions, average='binary') * 100, 4)
    mcc       = round(matthews_corrcoef(y_test, predictions) * 100, 4)
    
    train_predictions = classifier.predict(X_train)
    train_predictions_proba = classifier.predict_proba(X_train)
    train_predictions_proba = train_predictions_proba[:, 1]
    
    train_accuracy  = round(accuracy_score(y_train, train_predictions) * 100, 4)
    train_auc       = round(roc_auc_score(y_train, train_predictions_proba) * 100, 4)
    train_aupr      = round(average_precision_score(y_train, train_predictions_proba) * 100, 4)
    train_precision = round(precision_score(y_train, train_predictions, average='binary') * 100, 4)
    train_recall    = round(recall_score(y_train, train_predictions, average='binary') * 100, 4)
    train_f1        = round(f1_score(y_train, train_predictions, average='binary') * 100, 4)
    train_mcc       = round(matthews_corrcoef(y_train, train_predictions) * 100, 4)
    
    
    return accuracy, auc, aupr, precision, recall, f1, mcc, fp, tp, y_test, predictions_proba, train_accuracy, train_auc, train_aupr, train_precision, train_recall, train_f1, train_mcc, y_train, train_predictions_proba

# Function for computing independent test set ##

In [ ]:
def independent_test_scores(classifier, X_train, X_test, y_train, y_test):
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    predictions_proba = classifier.predict_proba(X_test)
    predictions_proba = predictions_proba[:, 1]
    
    fp, tp, threshold = roc_curve(y_test, predictions_proba)
    precision_position, recall_position, _ = precision_recall_curve(y_test, predictions_proba)
    
    accuracy  = round(accuracy_score(y_test, predictions) * 100, 4)
    auc       = round(roc_auc_score(y_test, predictions_proba) * 100, 4)
    aupr      = round(average_precision_score(y_test, predictions_proba) * 100, 4)
    precision = round(precision_score(y_test, predictions, average='binary') * 100, 4)
    recall    = round(recall_score(y_test, predictions, average='binary') * 100, 4)
    f1        = round(f1_score(y_test, predictions, average='binary') * 100, 4)
    mcc       = round(matthews_corrcoef(y_test, predictions) * 100, 4)
        
    return accuracy, auc, aupr, precision, recall, f1, mcc, fp, tp, y_test, predictions_proba

## Loading the feature generated dataset ##

In [ ]:
df = pd.read_csv("Dataset.csv")
df = df.iloc[:, : df.shape[1] - 1] 
df.head()

## Data preprocessing - scaling the data of selected features between 0 - 1 ##

In [ ]:
X = df.iloc[0:, 2:4]
df.iloc[0:, 2:4] = MinMaxScaler().fit_transform(X)
X = np.array(df.iloc[0:, 9632]).reshape(-1, 1)
df.iloc[0:, 9632] = MinMaxScaler().fit_transform(X)
X = np.array(df.iloc[0:, 9633]).reshape(-1, 1)
df.iloc[0:, 9633] = MinMaxScaler().fit_transform(X)
X = np.array(df.iloc[0:, 9634]).reshape(-1, 1)
df.iloc[0:, 9634] = MinMaxScaler().fit_transform(X)
X = np.array(df.iloc[0:, 9635]).reshape(-1, 1)
df.iloc[0:, 9635] = MinMaxScaler().fit_transform(X)
X = np.array(df.iloc[0:, 9636]).reshape(-1, 1)
df.iloc[0:, 9636] = MinMaxScaler().fit_transform(X)
df.head()

## Diving dataset into X and y ##

In [ ]:
X = df.iloc[0:, 2:]
y = df.iloc[0:, 0]

# Regression to Classification

In [ ]:
for i in range(len(y)):
    if float(y[i]) > 0.626:
        y[i] = 1
    else:
        y[i] = 0

# Train Index List and Test Index List (Test Size = 10%)

In [ ]:
train = [4955, 1412, 2873, 308, 4886, 624, 2575, 4402, 3200, 2236, 3150, 1115, 3119, 2259, 1200, 4835, 2053, 1421, 1188, 2846, 1128, 3799, 952, 1223, 856, 438, 696, 584, 3839, 2342, 149, 4977, 437, 12, 93, 3674, 2228, 705, 2276, 4071, 805, 1212, 2292, 642, 1101, 734, 1561, 1370, 2380, 2370, 4265, 429, 848, 4017, 4379, 2167, 2024, 3305, 3282, 2005, 1094, 2347, 3961, 3447, 2715, 2651, 2225, 4985, 4531, 287, 3527, 2456, 1780, 3828, 561, 4003, 1879, 3929, 798, 3159, 3792, 4275, 4960, 5210, 1509, 3048, 1730, 5265, 4351, 4720, 2072, 4078, 4728, 2627, 5072, 4870, 2741, 1498, 751, 2080, 1583, 1665, 1751, 1411, 5145, 414, 3666, 3820, 2339, 3313, 3402, 4354, 2995, 252, 3614, 654, 350, 4302, 877, 457, 2531, 4817, 90, 5005, 5289, 3442, 26, 549, 181, 2517, 3700, 2121, 2384, 2077, 2742, 1803, 776, 4366, 4579, 3088, 4934, 3161, 5146, 4655, 1897, 3112, 69, 2394, 3095, 3788, 5217, 926, 2647, 203, 3349, 339, 1361, 387, 2523, 4946, 4098, 4345, 4288, 2373, 810, 3949, 3132, 221, 677, 2462, 4332, 4498, 2723, 4475, 1095, 2464, 4382, 2855, 4816, 177, 4266, 2905, 2833, 1345, 3905, 3714, 931, 1746, 1942, 2943, 3107, 1593, 4635, 4050, 2399, 527, 3694, 1564, 881, 2099, 681, 794, 5042, 2091, 4865, 2104, 2181, 2973, 2369, 2552, 219, 3153, 1557, 582, 4706, 4310, 43, 4727, 210, 3433, 354, 2827, 3947, 1055, 4252, 3070, 367, 2592, 4661, 1891, 135, 4397, 47, 718, 1074, 1244, 2917, 4857, 670, 3106, 5275, 1044, 61, 1860, 4073, 95, 1736, 478, 121, 2892, 4633, 1744, 1543, 3708, 1360, 1032, 693, 1513, 322, 1400, 2209, 3661, 2662, 2499, 4393, 2093, 1503, 5011, 1419, 1658, 3914, 731, 334, 291, 184, 4640, 2857, 4702, 533, 1894, 1623, 1886, 4011, 1334, 5284, 657, 3323, 5074, 4567, 2726, 2057, 2810, 428, 1235, 4333, 2022, 5014, 3078, 1617, 1553, 3100, 4725, 977, 2685, 1702, 3235, 4513, 1961, 4726, 1732, 3245, 3227, 1041, 1126, 4808, 5308, 879, 4139, 1424, 3360, 5193, 1073, 764, 4462, 1554, 1448, 2525, 4284, 1292, 4764, 2170, 102, 1321, 1966, 4399, 5158, 3338, 1533, 2303, 2497, 70, 1176, 1391, 4489, 2364, 2311, 4166, 831, 1268, 4990, 1374, 3575, 969, 1456, 896, 4880, 1745, 2302, 4512, 3652, 4100, 4815, 4370, 3405, 1215, 1672, 1534, 4409, 3617, 5173, 4441, 2673, 5182, 381, 1747, 4851, 3682, 4377, 5261, 3491, 4454, 2117, 1322, 4065, 3801, 233, 712, 3844, 4849, 3481, 633, 4543, 415, 3722, 2605, 4892, 4950, 3931, 757, 3076, 605, 17, 4621, 2344, 2463, 3800, 596, 1832, 4949, 4446, 1867, 3613, 1741, 2101, 371, 2655, 2687, 652, 598, 676, 238, 3848, 3933, 1197, 3650, 5206, 2968, 1025, 5096, 75, 1626, 4527, 3917, 3513, 3837, 3109, 1297, 1231, 3689, 1773, 4519, 3827, 828, 1972, 4860, 96, 2409, 4915, 2721, 1467, 1743, 621, 2594, 4651, 4210, 566, 1813, 4074, 1487, 748, 1476, 3815, 51, 889, 1822, 2998, 2954, 1616, 534, 745, 3601, 2052, 443, 1210, 195, 4582, 2602, 3014, 802, 3252, 5268, 3396, 4683, 3790, 3825, 3908, 4287, 1323, 1181, 644, 1514, 3886, 120, 2312, 1839, 4597, 4278, 2186, 2765, 4209, 2570, 134, 3578, 1965, 511, 62, 4986, 1330, 1338, 2939, 122, 5250, 3253, 2882, 4394, 567, 1921, 3603, 864, 3967, 5099, 1840, 2481, 720, 3058, 2288, 2714, 602, 1220, 3208, 2767, 4235, 4447, 3906, 318, 274, 4580, 1183, 1632, 1260, 2138, 4713, 5125, 295, 1890, 3268, 298, 68, 3238, 3489, 1319, 150, 756, 4966, 4663, 1335, 2233, 1406, 376, 5204, 859, 3210, 4954, 378, 1807, 4229, 2847, 3410, 2826, 841, 2029, 175, 3555, 927, 2591, 45, 1096, 4414, 4253, 5199, 1650, 4866, 2378, 283, 2495, 1877, 2585, 2098, 2457, 1295, 1864, 2414, 4781, 1580, 2518, 3038, 3870, 1666, 4842, 257, 3732, 5092, 99, 4878, 1849, 3114, 1272, 3679, 1881, 366, 2333, 3469, 4122, 2865, 949, 1934, 19, 1630, 668, 218, 2881, 2187, 3716, 3881, 3121, 2516, 777, 2235, 1018, 2929, 1618, 3438, 517, 5067, 2580, 2164, 5100, 2659, 2316, 4844, 2656, 2389, 1052, 1425, 759, 3010, 5298, 214, 2388, 4328, 1907, 4294, 3591, 1538, 196, 4031, 5215, 4075, 1056, 1837, 887, 1657, 3765, 2803, 568, 4010, 2890, 2794, 139, 1504, 1149, 3488, 4497, 2577, 463, 2629, 4471, 594, 3318, 450, 3656, 2834, 2862, 2987, 1480, 1520, 1174, 5012, 451, 4476, 3372, 3303, 3651, 4187, 4974, 999, 3311, 2240, 192, 4623, 4787, 4443, 3895, 4967, 4704, 4163, 2885, 2226, 4660, 3921, 4456, 1669, 3000, 3355, 368, 1892, 1992, 2549, 2836, 5222, 191, 1815, 108, 3543, 2671, 3780, 3680, 2242, 2480, 1468, 3047, 476, 4700, 4037, 312, 5301, 166, 1983, 2002, 1024, 1221, 1770, 3240, 4784, 1765, 4984, 1536, 3334, 2157, 4746, 1988, 1541, 2223, 4916, 152, 5084, 2859, 4981, 4290, 1818, 1436, 5053, 3231, 1344, 5121, 4836, 1945, 31, 1225, 937, 4941, 752, 3557, 309, 2771, 4035, 3882, 2372, 495, 3434, 205, 1135, 3013, 3795, 701, 305, 1192, 4194, 4231, 1061, 843, 3142, 2543, 211, 1108, 4148, 679, 565, 5030, 4044, 680, 180, 5138, 1501, 2614, 4541, 278, 4216, 3101, 1204, 1578, 432, 289, 1027, 535, 3878, 1187, 188, 2646, 5162, 4239, 706, 3697, 270, 4584, 4634, 3023, 4135, 111, 1170, 5018, 220, 4410, 627, 2224, 3794, 5040, 790, 5171, 461, 4680, 551, 4263, 1427, 1703, 198, 2921, 2555, 402, 5141, 2607, 5061, 2284, 2067, 4743, 650, 4259, 708, 4400, 4693, 1782, 4043, 1216, 5277, 3404, 314, 4547, 5307, 4755, 113, 960, 4574, 2550, 2698, 3753, 5309, 1957, 3997, 1788, 3862, 4811, 4463, 2154, 2218, 4831, 326, 604, 4453, 3413, 5101, 4605, 4913, 2822, 1634, 3229, 1859, 4406, 1760, 2092, 2620, 789, 3487, 1550, 4246, 871, 5071, 5133, 5047, 2153, 73, 420, 1752, 1786, 3406, 92, 4858, 4589, 2533, 3258, 4019, 2100, 494, 1375, 3544, 4181, 2422, 1164, 2111, 1270, 3116, 4822, 5207, 2694, 618, 4561, 4309, 124, 2078, 3384, 5036, 1878, 4144, 2328, 3858, 869, 3066, 1042, 3017, 356, 1162, 4036, 3123, 3407, 4554, 1919, 4919, 5167, 4549, 4109, 3193, 5110, 332, 1599, 3090, 1995, 1258, 4576, 1451, 4207, 1116, 4906, 4016, 2841, 4834, 3691, 1694, 445, 2535, 2837, 1413, 452, 3467, 4714, 3337, 14, 4174, 945, 247, 4517, 360, 4945, 4975, 3866, 2600, 1586, 3585, 4524, 3361, 3946, 2450, 5188, 3220, 490, 3877, 862, 1006, 4161, 1383, 2191, 2763, 2927, 1362, 3618, 506, 1417, 3365, 25, 2642, 5266, 1222, 2251, 1113, 4618, 5039, 3871, 1670, 4964, 3353, 433, 2321, 3570, 3199, 2732, 3001, 4625, 2815, 3145, 4304, 1102, 2559, 2992, 2406, 4221, 156, 3366, 194, 615, 643, 2916, 3518, 1068, 1808, 5045, 3712, 5243, 4505, 3276, 330, 3154, 4130, 2527, 622, 3647, 4102, 3867, 3277, 4490, 2683, 4291, 4307, 755, 518, 4296, 32, 3316, 4189, 2856, 1157, 1738, 744, 1729, 4509, 2997, 4347, 4766, 783, 3196, 2829, 229, 857, 2974, 1401, 2663, 408, 170, 540, 1723, 4172, 4257, 3803, 613, 2346, 4435, 3887, 1161, 2314, 3298, 2280, 3225, 2004, 1609, 3861, 4896, 564, 4034, 5216, 2142, 1820, 4599, 1700, 4803, 5234, 4692, 3951, 2515, 2441, 2699, 2805, 5130, 1172, 4545, 325, 1051, 3326, 5083, 3260, 1789, 4502, 1196, 620, 1691, 4889, 1482, 1189, 328, 648, 1862, 897, 1817, 1123, 5060, 4930, 4552, 1726, 3002, 2217, 1779, 3912, 2416, 2874, 1519, 2900, 1874, 3703, 1023, 4710, 2937, 3503, 4864, 4420, 3320, 809, 3386, 1675, 978, 5306, 1629, 3035, 430, 4876, 2812, 3752, 2638, 2867, 4649, 2776, 2753, 174, 2705, 4419, 3254, 4742, 217, 4722, 1721, 4151, 1444, 690, 2115, 4127, 2521, 2526, 1106, 2447, 818, 4450, 611, 3248, 3667, 1697, 136, 2025, 2572, 576, 2317, 3026, 4716, 1180, 888, 3541, 324, 4528, 817, 3704, 2182, 208, 293, 4006, 1119, 2498, 4059, 1264, 990, 2569, 1614, 3204, 1739, 4460, 4536, 1662, 353, 4320, 4020, 1844, 1654, 5015, 1489, 2883, 3213, 3450, 1228, 259, 2622, 2981, 5035, 4510, 2868, 52, 4957, 411, 4149, 4245, 3148, 2643, 2910, 2436, 1479, 1507, 1033, 1097, 1469, 543, 1046, 2952, 1954, 30, 725, 2748, 5296, 4125, 5230, 1426, 2059, 2818, 436, 2291, 5189, 912, 4486, 2215, 1588, 5303, 4794, 2755, 4445, 1590, 3999, 1971, 1238, 903, 2127, 3559, 3125, 507, 4586, 183, 2118, 1497, 449, 321, 1211, 4066, 5081, 2119, 1769, 4214, 2133, 3767, 2172, 2377, 1551, 4718, 1146, 3126, 2877, 44, 1313, 2018, 1893, 2728, 3044, 3771, 1378, 2387, 3409, 413, 4999, 1407, 941, 801, 1288, 1558, 2458, 4676, 4546, 4801, 1699, 256, 4901, 5211, 1151, 2598, 647, 1767, 3285, 4430, 2086, 2194, 2083, 24, 1315, 1088, 2252, 2784, 3873, 1350, 3658, 112, 3547, 263, 1393, 4689, 1186, 4827, 5028, 56, 1309, 695, 4466, 4479, 2658, 2780, 1227, 276, 2107, 319, 1833, 3918, 1589, 1858, 682, 2488, 3692, 3634, 479, 3964, 3221, 286, 2760, 3425, 3307, 2727, 4133, 2094, 3956, 2486, 1598, 2146, 5113, 3868, 736, 2222, 336, 1872, 5200, 3129, 3392, 3670, 5069, 1071, 2586, 2124, 4201, 3773, 2173, 3492, 4374, 81, 4381, 3809, 1714, 1488, 1777, 498, 4615, 3475, 2886, 1946, 4099, 3299, 3439, 4093, 4128, 1610, 4871, 2574, 4195, 2940, 1022, 3214, 5190, 2264, 4168, 3504, 2282, 665, 5103, 5001, 1552, 5144, 2437, 1017, 557, 3968, 2706, 1207, 2275, 3571, 2402, 1510, 3727, 2439, 1684, 245, 1163, 4262, 5183, 3246, 266, 3910, 4828, 1114, 63, 2337, 1909, 2924, 4342, 3113, 3408, 3482, 2845, 1876, 3480, 4040, 4255, 2063, 4072, 1784, 1237, 103, 1873, 2348, 2980, 272, 508, 4588, 5239, 5283, 908, 1303, 3592, 1627, 1522, 835, 358, 138, 299, 1556, 1003, 382, 4765, 761, 2876, 2341, 2478, 4303, 1402, 2304, 4573, 1932, 2299, 4241, 2244, 4566, 58, 76, 796, 4436, 785, 3034, 5212, 4885, 2670, 4110, 910, 2804, 5085, 1257, 3935, 3388, 1178, 3922, 442, 3243, 2440, 2619, 5159, 3919, 2114, 4069, 2417, 3707, 3485, 3403, 2678, 3437, 1502, 2471, 3064, 2011, 3973, 3321, 4292, 3955, 6, 678, 1943, 3331, 4894, 2722, 4087, 3378, 5172, 1064, 2977, 1474, 4540, 4250, 2751, 2021, 4604, 4940, 2819, 1130, 1461, 4812, 2688, 4595, 222, 4927, 48, 573, 1236, 4005, 4086, 4778, 267, 1104, 1935, 386, 3737, 4867, 4281, 4657, 1563, 4412, 209, 909, 1289, 3950, 3071, 1842, 4353, 4457, 958, 1793, 2088, 331, 2654, 3096, 4701, 1774, 2680, 3605, 4322, 2277, 3431, 2229, 4989, 4032, 1366, 173, 5102, 3764, 2298, 4641, 1952, 2899, 2036, 3807, 3728, 4152, 0, 2174, 1356, 3838, 2522, 5273, 396, 3911, 3134, 1812, 2879, 4371, 816, 1572, 3352, 1800, 4211, 4082, 2609, 4594, 3077, 2750, 3542, 3418, 5142, 554, 1159, 1213, 5031, 5294, 3940, 3169, 700, 4337, 3471, 3740, 1455, 4458, 3536, 3655, 866, 1644, 3593, 3187, 3957, 1490, 2451, 729, 3065, 3865, 1936, 1110, 4325, 4935, 1185, 2007, 193, 1515, 578, 2123, 1429, 4058, 5259, 1846, 131, 178, 4747, 3209, 3456, 3105, 2962, 2073, 658, 1896, 3846, 2221, 5126, 342, 4814, 727, 5186, 976, 2505, 3364, 1928, 4882, 3662, 1299, 3381, 4883, 660, 1790, 2700, 3657, 3362, 3927, 3576, 3050, 5241, 3829, 1278, 3473, 1621, 3411, 3194, 67, 5155, 2991, 2269, 4198, 820, 1226, 3383, 4451, 2842, 3098, 4738, 4537, 3786, 1143, 397, 4558, 4577, 3699, 1398, 1659, 2220, 979, 1298, 3460, 82, 3857, 5129, 4160, 4190, 2584, 423, 4609, 1904, 4821, 3907, 4731, 1565, 3793, 839, 500, 141, 1377, 4137, 2266, 3888, 1138, 4147, 4695, 1787, 2895, 550, 3509, 2019, 948, 2554, 2476, 532, 3062, 1829, 3295, 4697, 4238, 1602, 2238, 639, 5132, 967, 4503, 2960, 1683, 2248, 4751, 2641, 155, 2295, 3074, 3203, 1089, 4786, 2319, 4506, 288, 930, 2860, 2689, 3084, 4824, 3814, 2669, 3996, 942, 2354, 4228, 4953, 1698, 5002, 3339, 3370, 3678, 3470, 985, 3130, 4644, 2512, 5075, 981, 2813, 2758, 4258, 3322, 3985, 5160, 4222, 1457, 1395, 264, 2702, 929, 3463, 5221, 3941, 2178, 4650, 246, 4982, 1758, 5166, 4504, 923, 2644, 268, 669, 3089, 1910, 3375, 1152, 5187, 3904, 4507, 1781, 2413, 2307, 2445, 3371, 1915, 612, 4335, 2267, 3988, 1898, 1649, 3725, 258, 3757, 847, 4888, 4518, 1233, 1127, 3085, 1967, 558, 904, 484, 2211, 1320, 2653, 7, 2315, 3706, 3607, 1450, 1201, 2426, 1814, 773, 2274, 1011, 3181, 2764, 1991, 1277, 1783, 3516, 446, 880, 4740, 781, 1749, 3834, 2932, 2423, 5184, 4591, 883, 3319, 1091, 3939, 3317, 531, 4083, 2438, 4316, 3755, 4358, 2290, 1727, 610, 570, 3011, 3426, 343, 4685, 3554, 434, 1903, 97, 3263, 2069, 1537, 2792, 4672, 1517, 965, 4684, 528, 176, 2470, 3937, 3033, 5037, 572, 2596, 3965, 4418, 2778, 2374, 3632, 3986, 2208, 2894, 2492, 3954, 383, 852, 1880, 370, 4254, 5131, 4593, 1105, 2902, 2361, 4279, 1336, 2189, 4877, 3774, 2617, 3758, 3552, 5148, 1171, 2925, 3039, 1422, 2640, 2988, 4417, 1464, 874, 2297, 2623, 4610, 1251, 2648, 840, 4217, 581, 204, 1641, 2014, 905, 631, 3501, 1974, 162, 3589, 3784, 4952, 2801, 617, 875, 1431, 547, 3768, 4829, 4487, 2692, 4386, 3274, 4340, 1861, 640, 3435, 637, 4847, 3909, 2308, 4313, 2096, 4041, 3479, 1372, 3206, 921, 546, 1449, 2610, 1900, 2028, 2301, 1885, 1642, 3966, 5108, 2097, 4628, 4025, 4776, 5274, 836, 601, 2184, 4232, 2802, 4575, 1582, 4534, 2858, 3075, 1070, 1775, 5107, 1283, 4270, 3702, 5050, 1836, 4508, 1019, 3626, 1326, 475, 3897, 1651, 4951, 4553, 2528, 2944, 4570, 2915, 1373, 739, 4173, 2754, 867, 1140, 997, 1337, 541, 1099, 1132, 5152, 3063, 2258, 4299, 59, 3256, 1938, 2908, 5242, 2704, 4758, 1078, 3296, 3540, 4532, 2630, 2809, 940, 1080, 187, 2970, 2085, 2249, 2941, 5195, 2023, 3094, 1492, 4076, 226, 3733, 2583, 1929, 3458, 4771, 1404, 2179, 697, 1124, 3158, 4280, 1778, 2634, 3226, 1037, 4437, 2256, 5247, 313, 2649, 2281, 3312, 2547, 1416, 1036, 2159, 4425, 1754, 3525, 3176, 3045, 2520, 3628, 3781, 3879, 1506, 1923, 448, 2757, 2851, 5276, 2893, 361, 2183, 1134, 4329, 2155, 1830, 1053, 2460, 22, 2066, 1653, 4396, 3631, 2743, 4009, 4326, 2468, 2325, 460, 1067, 87, 1430, 2984, 282, 2483, 462, 212, 2588, 2820, 2581, 1368, 234, 3128, 597, 4699, 2250, 4522, 2283, 662, 5270, 2285, 5010, 3611, 5251, 4482, 3880, 779, 2544, 3819, 3874, 3715, 3616, 5135, 685, 3586, 3102, 3883, 2171, 3007, 1850, 487, 1688, 2230, 3690, 2320, 782, 521, 522, 3811, 4283, 2398, 2207, 4054, 4013, 4159, 3185, 1511, 1882, 651, 3346, 3497, 369, 4648, 5097, 118, 3751, 5033, 2216, 4002, 1613, 3290, 962, 2510, 4008, 4909, 4169, 3494, 2415, 4942, 4501, 2185, 3184, 2567, 2037, 4529, 772, 1284, 5229, 5032, 2587, 5227, 1414, 1234, 4273, 435, 1674, 365, 2986, 231, 4535, 147, 4675, 3270, 407, 2090, 86, 844, 1031, 4352, 3622, 3970, 3654, 2628, 4088, 855, 78, 2247, 3515, 4853, 4890, 758, 4047, 3286, 2279, 4145, 4421, 4237, 4378, 2176, 424, 392, 5020, 4991, 104, 2672, 3358, 3889, 3550, 3172, 1948, 1459, 3683, 3166, 3053, 5088, 1587, 5290, 1731, 5062, 4372, 182, 1791, 2355, 1940, 702, 2442, 4376, 496, 464, 4624, 4603, 172, 3477, 5095, 2540, 3688, 3333, 2260, 2112, 900, 1446, 4038, 2697, 3630, 741, 2293, 2452, 2957, 4346, 1206, 509, 4416, 2955, 3165, 5149, 1549, 3693, 2202, 2566, 4724, 2044, 767, 5175, 1508, 421, 123, 4424, 3749, 819, 1403, 1382, 2148, 4483, 1575, 1638, 1795, 5114, 4733, 399, 2310, 882, 5280, 3273, 1352, 4861, 4833, 2130, 982, 456, 3989, 1079, 1950, 4666, 689, 2429, 641, 2392, 2825, 1953, 3103, 3382, 5223, 2710, 3770, 5064, 3309, 1884, 3915, 389, 2494, 3806, 4606, 845, 542, 91, 4997, 1979, 4980, 4349, 3279, 2964, 2529, 1676, 1048, 2949, 1960, 2530, 3750, 4769, 237, 3351, 3944, 1133, 2880, 1805, 4404, 163, 3948, 5055, 5003, 3779, 3902, 3619, 265, 2001, 1811, 2978, 4760, 2906, 4049, 1428, 4084, 3162, 5245, 4741, 4937, 5063, 1452, 593, 2725, 41, 560, 3315, 3443, 243, 1339, 3139, 3742, 405, 1442, 261, 4600, 1689, 1725, 1203, 834, 1866, 2496, 3016, 4192, 2010, 3135, 340, 3379, 1601, 2796, 3398, 4987, 480, 3893, 4155, 1523, 3841, 4719, 4925, 4539, 799, 1766, 5023, 3625, 2453, 3428, 5236, 4311, 4969, 363, 1271, 3483, 4783, 4355, 4403, 347, 4244, 4908, 2562, 3027, 4873, 1423, 656, 3623, 4898, 2102, 632, 4846, 2493, 2665, 3638, 4182, 4091, 3505, 4063, 2500, 1465, 3197, 115, 1821, 1494, 2401, 1232, 2661, 2817, 3244, 4856, 1153, 4405, 57, 3842, 3624, 4556, 5252, 1735, 2823, 2196, 4015, 2475, 2564, 3574, 5065, 2608, 4970, 707, 3046, 4899, 1093, 4024, 1376, 5219, 72, 1922, 3590, 1764, 743, 3250, 1005, 2668, 1190, 3608, 2601, 1125, 1241, 3217, 4391, 3891, 998, 3832, 2684, 1577, 5089, 1826, 3721, 5297, 3994, 440, 5258, 1100, 2637, 4673, 649, 1546, 3998, 4022, 3580, 3604, 4162, 630, 672, 1249, 3259, 482, 1584, 3478, 4912, 5156, 1462, 3040, 4269, 4094, 5288, 3131, 2578, 4196, 49, 4557, 3459, 858, 1072, 719, 3573, 2631, 5176, 1660, 1656, 2425, 3982, 5049, 5281, 3928, 3097, 4757, 3734, 5048, 3189, 2934, 4387, 3519, 2606, 1385, 5123, 1399, 1195, 1408, 1845, 1798, 1475, 3639, 35, 4001, 2345, 4264, 2359, 3006, 2206, 3620, 4563, 3179, 1286, 349, 2137, 2691, 4369, 3538, 1681, 4315, 1443, 3669, 3215, 1671, 3336, 1179, 2798, 1692, 4240, 372, 3055, 4080, 1916, 1357, 1825, 786, 469, 140, 788, 323, 50, 714, 4770, 2840, 2446, 5112, 3567, 4698, 4879, 3412, 2666, 406, 4368, 1004, 3400, 890, 2411, 2545, 3885, 2390, 5009, 42, 2830, 3739, 3022, 303, 4795, 11, 664, 1177, 3292, 2676, 1947, 2501, 1473, 3938, 3978, 2132, 1026, 2989, 961, 2889, 3021, 3015, 3548, 2009, 2149, 3754, 2338, 735, 304, 1440, 1496, 2652, 2875, 916, 4444, 710, 4910, 4690, 2772, 1624, 4220, 821, 3729, 1354, 5153, 850, 3350, 1000, 3821, 18, 1637, 1667, 493, 3173, 1347, 3018, 4154, 3230, 4004, 4085, 3936, 687, 1112, 3232, 1608, 4350, 4205, 1009, 3291, 2408, 1092, 1327, 4947, 1677, 1999, 1310, 3615, 3455, 3684, 1987, 4112, 344, 4448, 986, 587, 2791, 4581, 3520, 4305, 3499, 3429, 3289, 1142, 2358, 3264, 1198, 85, 1518, 516, 1381, 2716, 4478, 1014, 128, 1364, 4096, 4293, 3537, 891, 2793, 4472, 1359, 307, 1661, 2431, 4274, 1317, 4678, 3249, 2474, 2542, 5124, 1246, 5057, 1208, 3377, 216, 4560, 3916, 1239, 2289, 2740, 275, 489, 1035, 1229, 4868, 1008, 5000, 3535, 3599, 2532, 4431, 1058, 2166, 4359, 3816, 281, 4028, 1316, 5285, 968, 3539, 1939, 3583, 3977, 619, 3660, 1742, 1390, 4334, 1847, 1579, 1655, 2352, 2212, 3275, 2020, 5279, 1956, 4046, 3192, 1307, 2003, 5093, 575, 2360, 4939, 4717, 1706, 1799, 3347, 4230, 1280, 4331, 861, 4407, 1801, 1628, 1993, 2639, 269, 629, 3081, 3466, 2334, 4647, 2966, 300, 459, 1447, 1472, 2046, 4818, 4180, 4060, 2113, 27, 2040, 4805, 4434, 837, 129, 3808, 2008, 5213, 503, 4739, 4572, 380, 1851, 1525, 2365, 3653, 3052, 2884, 2340, 374, 3686, 2381, 4578, 939, 5090, 4226, 4427, 2799, 2147, 4401, 4823, 4480, 2068, 529, 3308, 5143, 636, 1269, 185, 1394, 4559, 3903, 1797, 2979, 1441, 3875, 5177, 1535, 3892, 3188, 1273, 2467, 607, 3833, 1712, 2199, 955, 2579, 3802, 4256, 4705, 3824, 2852, 403, 2071, 5052, 1977, 158, 1748, 2909, 3992, 902, 352, 1230, 4961, 2140, 444, 5255, 754, 1737, 2971, 5286, 2920, 311, 3423, 548, 661, 5059, 1989, 306, 2335, 694, 2356, 1927, 684, 3427, 749, 3945, 4918, 1266, 5194, 5046, 4703, 5201, 2134, 3083, 3061, 3025, 970, 13, 873, 808, 3198, 514, 4753, 409, 3856, 5291, 5248, 4679, 207, 4053, 458, 4686, 2430, 250, 110, 4167, 1107, 4317, 1883, 1713, 1287, 4968, 3441, 552, 1439, 2270, 4103, 3144, 2821, 5079, 1925, 634, 5104, 316, 4481, 4993, 1328, 427, 3817, 105, 3041, 142, 2169, 3761, 89, 2918, 3237, 2241, 3174, 3860, 2163, 2636, 1949, 1768, 3711, 2272, 1, 3710, 5264, 1951, 935, 4564, 2720, 3942, 1109, 4224, 591, 3534, 1695, 3899, 2936, 4467, 3859, 1576, 1111, 213, 778, 1415, 3496, 101, 2043, 2848, 4820, 4656, 4204, 1567, 165, 5008, 3566, 4048, 5232, 1343, 2013, 273, 71, 4251, 1087, 394, 583, 1367, 5295, 4963, 2548, 1013, 5203, 3003, 5087, 1701, 4197, 1763, 1875, 1259, 3344, 4813, 2253, 2781, 5091, 526, 4213, 3869, 2946, 377, 1559, 692, 1963, 5027, 348, 2434, 2246, 4428, 3633, 1709, 1050, 590, 519, 3380, 3676, 4775, 4384, 2477, 1083, 4056, 3610, 5076, 4791, 1294, 1809, 3595, 791, 792, 4375, 1855, 5054, 1980, 3233, 3598, 933, 3831, 3328, 1530, 1410, 3524, 4562, 4936, 83, 868, 2350, 20, 3872, 4286, 4108, 2795, 3453, 4855, 3778, 1562, 4715, 3093, 3155, 973, 1710, 3201, 4295, 5118, 1098, 4423, 2844, 886, 5237, 4129, 3560, 4863, 5161, 4677, 515, 4905, 737, 1453, 3553, 3609, 2006, 4994, 2506, 2994, 2126, 2926, 3717, 5041, 906, 28, 1030, 2507, 2428, 947, 4200, 762, 3747, 715, 284, 686, 703, 3341, 2353, 2257, 4012, 473, 21, 2752, 885, 2448, 917, 2571, 1040, 2239, 730, 3395, 1265, 483, 4607, 2963, 1219, 277, 988, 2084, 1245, 2382, 571, 1435, 993, 3901, 3896, 585, 3526, 4116, 2514, 1349, 1007, 674, 2412, 2903, 447, 3926, 2913, 4141, 359, 2064, 4730, 4203, 1690, 4596, 1548, 1247, 1985, 294, 1547, 2012, 588, 3789, 3329, 2403, 2129, 4662, 3741, 3508, 2410, 2719, 4837, 425, 2582, 1054, 4839, 4845, 556, 4620, 4629, 2054, 5006, 2469, 3738, 5004, 355, 1193, 1716, 3416, 2604, 4390, 2828, 1704, 2863, 4601, 2618, 2400, 2139, 3239, 2383, 3218, 4165, 1285, 4538, 1772, 4749, 3930, 2087, 1889, 3963, 1591, 1346, 4711, 2162, 2396, 317, 2271, 4772, 3111, 1137, 592, 972, 795, 3020, 1711, 3387, 36, 453, 913, 4348, 1199, 3972, 1274, 2375, 37, 974, 3195, 983, 3297, 4979, 1418, 3332, 1560, 39, 1486, 520, 169, 1355, 2785, 4104, 5044, 3720, 3612, 987, 2682, 4998, 4319, 1640, 1290, 3681, 1380, 1933, 579, 4484, 3160, 2030, 892, 5111, 780, 4113, 159, 2, 2263, 3216, 2109, 849, 2737, 2466, 4708, 1914, 1596, 1776, 1913, 1043, 5007, 1544, 2736, 726, 4902, 53, 1796, 3588, 3845, 1911, 2306, 2553, 2800, 936, 4388, 824, 5029, 3178, 3401, 1389, 3564, 5179, 884, 5066, 3646, 3932, 3731, 1607, 1594, 771, 2611, 3782, 3224, 2273, 1868, 1085, 10, 4081, 1325, 3390, 1463, 806, 4336, 4734, 4184, 4068, 2108, 4500, 5263, 2234, 3211, 963, 3415, 3960, 2864, 398, 2632, 5151, 4077, 2996, 704, 431, 4958, 292, 5168, 145, 4408, 2033, 4691, 1165, 3500, 2050, 3760, 2624, 4272, 4126, 2424, 1262, 1771, 3511, 271, 244, 2032, 813, 3810, 4323, 1804, 2461, 545, 3241, 738, 310, 3521, 1131, 5246, 4111, 2769, 920, 3587, 3959, 38, 2983, 5299, 2667, 3223, 1182, 614, 5214, 1466, 3042, 934, 2419, 4797, 4773, 826, 1521, 2930, 628, 3853, 390, 5178, 3990, 400, 3359, 46, 5098, 5228, 3642, 3726, 5157, 2421, 1750, 5137, 4356, 4062, 338, 215, 3663, 1341, 4862, 3894, 3640, 329, 3004, 5080, 3115, 5140, 2322, 3549, 1458, 171, 4459, 3569, 4339, 4383, 938, 60, 1240, 4449, 2120, 1524, 153, 481, 3920, 1762, 750, 1785, 2711, 4301, 4193, 1333, 4732, 2854, 186, 1136, 2058, 4461, 148, 4124, 3363, 2866, 3776, 1386, 5025, 3629, 4785, 3028, 2969, 4627, 1708, 2953, 454, 609, 4515, 2074, 4052, 4327, 1622, 4664, 40, 2089, 3924, 2175, 74, 4260, 5235, 2657, 3180, 2076, 441, 2051, 2541, 1301, 3430, 1304, 236, 4438, 2599, 2573, 3280, 3186, 3953, 2768, 4223, 2106, 2679, 2761, 1387, 4988, 4809, 1279, 5205, 1555, 3190, 4983, 3594, 1920, 513, 4177, 1308, 914, 3452, 3695, 4242, 3156, 2198, 774, 943, 822, 4277, 164, 3983, 2538, 5073, 1311, 4830, 4439, 4932, 3300, 3310, 327, 2490, 4978, 832, 3079, 3462, 4665, 54, 4064, 3451, 1901, 1481, 4150, 3335, 946, 3293, 2161, 4511, 3057, 768, 4996, 3495, 285, 5244, 4796, 2332, 2717, 2079, 3389, 722, 951, 4638, 2262, 989, 3212, 1759, 4367, 1724, 341, 3701, 4243, 688, 3432, 966, 5021, 2203, 2713, 4687, 3572, 1990, 4120, 4362, 5038, 3600, 4850, 5287, 4385, 385, 4643, 4852, 536, 4956, 4212, 3507, 201, 1332, 3502, 2708, 4021, 1854, 4750, 2872, 3030, 1205, 1645, 3417, 3476, 2861, 1792, 3797, 3368, 3367, 3533, 4669, 3449, 603, 3056, 4631, 126, 3876, 2935, 3531, 2730, 827, 1639, 984, 3281, 5278, 2188, 4897, 2443, 362, 2349, 345, 814, 3205, 4422, 5094, 3325, 4106, 2116, 1834, 793, 5253, 2674, 94, 4225, 3756, 2891, 2371, 4411, 4712, 3228, 2853, 2912, 1865, 1857, 2645, 2243, 2336, 950, 3068, 1120, 2597, 2204, 419, 922, 1254, 477, 2035, 1202, 2626, 5117, 1970, 3110, 4018, 2151, 3687, 1997, 4033, 3054, 1248, 2192, 2712, 992, 2190, 3762, 3597, 1318, 1863, 2343, 2449, 2136, 1065, 5267, 4227, 2621, 77, 2898, 1314, 4645, 872, 3850, 5086, 2965, 3391, 2560, 5202, 2635, 3530, 375, 1908, 4429, 4602, 4652, 4027, 1604, 1118, 3677, 3490, 5165, 2958, 1994, 5262, 4626, 2376, 1342, 320, 1619, 3713, 2060, 2219, 1673, 260, 4696, 2039, 901, 3744, 2026, 4234, 1917, 1828, 1573, 4415, 4670, 4208, 224, 723, 4767, 1141, 2762, 846, 4289, 2551, 2928, 117, 645, 3, 4170, 467, 5, 919, 2595, 4324, 1063, 2590, 242, 1160, 2870, 3735, 2397, 3517, 3421, 2563, 1605, 5128, 2324, 2709, 4674, 1312, 3448, 3812, 4943, 2362, 1975, 3514, 1761, 2739, 4948, 2472, 4007, 1493, 4206, 2158, 1169, 4914, 232, 66, 1603, 3283, 3596, 125, 1296, 1976, 4800, 4763, 2075, 3086, 4616, 3345, 4745, 2797, 918, 1646, 364, 3287, 2125, 4123, 4569, 525, 55, 638, 302, 2766, 1276, 2055, 4432, 3644, 4143, 2878, 4202, 2286, 2261, 2296, 1753, 1968, 4622, 3117, 2393, 523, 3900, 3122, 4156, 1969, 671, 4188, 666, 4608, 1887, 3266, 280, 1719, 2947, 4455, 899, 5150, 760, 1906, 4042, 3823, 4138, 3818, 4300, 2593, 4164, 3031, 3579, 2832, 3472, 137, 2180, 2128, 3177, 4639, 4261, 4344, 1331, 2017, 4542, 2482, 1358, 1329, 2150, 4530, 2122, 3981, 1252, 3278, 3374, 2814, 1848, 3037, 388, 235, 1077, 1984, 4363, 3962, 770, 2165, 4533, 673, 249, 1574, 3419, 2081, 1823, 4917, 4544, 716, 4959, 2048, 4178, 2724, 4804, 1460, 1668, 2756, 2000, 4571, 4613, 404, 384, 4774, 2152, 1633, 4944, 3925, 4158, 223, 2193, 3136, 4469, 1824, 4884, 797, 2871, 9, 1944, 1838, 255, 4485, 4790, 1581, 2956, 924, 4709, 2508, 3775, 4136, 2779, 539, 2745, 4971, 4854, 1145, 4468, 667, 3019, 357, 2831, 2783, 1353, 470, 2703, 3330, 3637, 1396, 3584, 4768, 3376, 1256, 3512, 1250, 5106, 4298, 3284, 2933, 4175, 1852, 5170, 2105, 133, 928, 823, 4826, 1531, 1214, 504, 4285, 1996, 3072, 1958, 1384, 894, 455, 4923, 3709, 4903, 4748, 4268, 3522, 4134, 3602, 713, 4186, 2214, 2923, 1810, 3486, 3141, 1827, 1155, 1324, 4312, 510, 1680, 499, 4819, 1856, 1388, 3719, 2386, 3236, 1540, 1217, 2681, 2993, 1191, 1542, 3301, 3324, 2539, 953, 4330, 732, 1156, 4922, 3294, 2982, 2156, 1062, 2616, 1167, 3222, 3636, 2938, 4491, 3140, 143, 3577, 2782, 717, 1819, 2502, 4360, 4171, 395, 5136, 4492, 1365, 4841, 595, 3952, 2914, 2326, 3766, 3440, 3851, 4132, 4874, 2143, 1571, 4079, 1122, 2786, 2967, 2504, 3465, 675, 2808, 3348, 728, 412, 3146, 1516, 825, 3267, 3974, 616, 114, 417, 3069, 1625, 5257, 4185, 2896, 4962, 2082, 753, 2546, 3969, 3373, 4107, 2444, 4380, 1066, 3913, 3498, 2738, 4000, 1568, 2427, 3746, 4926, 1687, 851, 3546, 895, 4592, 335, 2839, 3087, 4611, 1348, 1015, 2816, 1631, 2489, 3730, 3251, 4920, 2070, 4452, 5185, 1635, 4646, 2479, 2265, 2959, 5026, 3059, 2141, 3118, 5019, 4045, 3356, 2744, 3506, 1679, 4249, 3147, 2294, 580, 1470, 4799, 4070, 3343, 4583, 3672, 559, 2849, 512, 4729, 1081, 1664, 1012, 1931, 1150, 2838, 116, 740, 5105, 3675, 830, 119, 4752, 5077, 5034, 635, 4465, 4364, 4142, 991, 876, 4869, 4, 3051, 3124, 2660, 2491, 3854, 4933, 4777, 1715, 1148, 3261, 574, 3769, 3736, 4146, 853, 1982, 2774, 4929, 1166, 971, 2701, 4495, 3847, 2368, 4115, 4780, 4131, 2950, 524, 2455, 1841, 537, 2806, 4341, 3175, 784, 4057, 1409, 2537, 154, 3009, 1686, 2901, 5256, 4721, 3863, 2733, 1895, 569, 1816, 2065, 127, 3394, 3327, 5293, 2975, 980, 2300, 190, 1139, 699, 5078, 606, 4276, 301, 3420, 5119, 1060, 2565, 5302, 1069, 2524, 2205, 1243, 16, 3120, 4642, 4754, 815, 3067, 5082, 959, 4893, 225, 3354, 2485, 562, 3436, 1693, 1045, 3446, 4565, 954, 5292, 2735, 5209, 4735, 5122, 1218, 1527, 3987, 5238, 1300, 5196, 160, 4067, 956, 3791, 3234, 2869, 3777, 4433, 4973, 2961, 2200, 98, 2557, 3183, 253, 3923, 4199, 1445, 1648, 4682, 4723, 3991, 1696, 3191, 4499, 1154, 2255, 1682, 3170, 2056, 4090, 3108, 4548, 3723, 2049, 4976, 3643, 5233, 623, 262, 3369, 4061, 1802, 4736, 5300, 2385, 1959, 2976, 488, 1843, 1734, 2911, 3219, 146, 1986, 1369, 2811, 659, 1794, 2511, 3397, 4282, 1147, 3272, 608, 3582, 1930, 197, 2931, 2327, 4389, 766, 4496, 4218, 3255, 202, 3510, 4931, 1733, 2777, 1707, 3474, 854, 4737, 4097, 2160, 698, 5254, 2999, 3696, 1636, 3457, 3581, 1291, 4659, 1853, 3763, 3304, 5218, 3743, 3445, 1998, 663, 4779, 3748, 4413, 4744, 709, 1282, 5068, 2675, 4029, 1306, 2237, 3302, 2038, 1529, 1663, 5240, 5260, 3627, 5147, 4891, 2693, 3060, 1597, 2729, 2625, 3157, 3314, 1722, 1059, 4636, 3242, 1678, 1757, 4191, 5116, 563, 2790, 863, 1484, 5134, 206, 4488, 870, 4493, 502, 1028, 3840, 2690, 5056, 1570, 2773, 2731, 804, 3645, 2454, 4598, 3104, 4875, 200, 3099, 2278, 3561, 4474, 4514, 391, 1495, 2695, 2027, 2568, 64, 3884, 2062, 492, 379, 4859, 3993, 4887, 1076, 878, 337, 1016, 1275, 3798, 4551, 3342, 4911, 995, 1981, 4297, 161, 4798, 1500, 3843, 5225, 2613, 3461, 1021, 3073, 3943, 1585, 1955, 3152, 34, 4014, 775, 2556, 1478, 1363, 2612, 2824, 2041, 241, 2061, 2363, 600, 2435, 2888, 646, 3890, 3556, 3202, 1528, 1267, 1899, 4658, 3005, 2734, 189, 1806, 975, 2747, 2047, 2558, 1082, 474, 2904, 4843, 4117, 3385, 4555, 1184, 5051, 2433, 2391, 769, 1685, 130, 2919, 3171, 3444, 4426, 466, 3092, 3772, 5191, 5226, 860, 4789, 4668, 1039, 2788, 5017, 1433, 3029, 4762, 3979, 2330, 3805]
test  = [5024, 1002, 2245, 1293, 1103, 3673, 2465, 4587, 1175, 3424, 3659, 2519, 3082, 1001, 240, 491, 829, 2770, 4585, 626, 3836, 865, 79, 1615, 2213, 1611, 472, 3759, 3606, 5304, 1437, 4612, 898, 5016, 5139, 3036, 4788, 2357, 2231, 1978, 3852, 3558, 555, 5127, 167, 4343, 8, 3357, 586, 4357, 530, 2850, 3091, 655, 3008, 3468, 1926, 2366, 2195, 3464, 1595, 1471, 4051, 15, 1144, 3785, 168, 911, 2131, 4318, 1090, 410, 4782, 2789, 538, 5164, 1491, 23, 2942, 2197, 393, 84, 3454, 2843, 2718, 3864, 2487, 907, 4924, 4140, 2664, 2177, 1224, 4308, 4761, 3049, 4516, 1255, 2034, 1420, 4030, 1569, 1158, 1545, 4179, 800, 3207, 5163, 4590, 1728, 4321, 2897, 3399, 1371, 4247, 2144, 3621, 4055, 625, 3043, 2395, 4637, 4119, 3164, 3835, 2432, 1010, 3562, 2135, 2650, 4667, 80, 4089, 3830, 4183, 3804, 1620, 2379, 4215, 1606, 4832, 2633, 5174, 4907, 2016, 3665, 2287, 4707, 4477, 5269, 1242, 501, 2513, 1505, 577, 3822, 3182, 2775, 29, 3288, 4681, 1392, 3745, 2686, 296, 297, 3971, 333, 5305, 2990, 2095, 230, 4938, 4568, 4442, 239, 1084, 1592, 3934, 4153, 107, 5282, 957, 3565, 4793, 132, 4092, 1034, 5198, 1121, 1117, 2603, 3826, 486, 1029, 4176, 5231, 3532, 4653, 742, 290, 1379, 2318, 157, 65, 2404, 3523, 1512, 1263, 2576, 3698, 418, 4101, 765, 373, 721, 3414, 833, 2031, 1964, 4525, 1831, 3568, 4105, 3340, 1962, 1526, 1038, 2696, 4526, 1168, 711, 1532, 2951, 2473, 4759, 1302, 1253, 1612, 3976, 3648, 807, 553, 2227, 151, 3143, 33, 1485, 4806, 2232, 279, 3813, 505, 2367, 2103, 1483, 1647, 346, 5070, 4271, 251, 5272, 3137, 465, 2210, 227, 1405, 422, 803, 3257, 5220, 1340, 653, 3705, 1756, 4900, 1718, 1351, 5022, 5197, 4792, 763, 1888, 351, 691, 1047, 3563, 724, 1432, 2787, 2534, 4395, 497, 4756, 3551, 2945, 2922, 5181, 1539, 3163, 4494, 2972, 4895, 4521, 106, 544, 915, 4470, 4338, 1905, 3545, 3855, 3138, 1281, 439, 4440, 2407, 2254, 1871, 3529, 3484, 2168, 3149, 3664, 179, 5271, 3671, 812, 4026, 1434, 4023, 5224, 1129, 4825, 4236, 1755, 4267, 4233, 401, 2042, 2309, 1941, 5120, 3265, 3032, 4881, 1499, 426, 2835, 3980, 1643, 3168, 3796, 2907, 2746, 4810, 2405, 1194, 4306, 4807, 2045, 4248, 893, 932, 2313, 3271, 2536, 4114, 1717, 2749, 2420, 1305, 1740, 4520, 3685, 3422, 5013, 3393, 254, 1438, 2418, 2887, 4840, 4965, 4121, 2615, 996, 1912, 964, 4694, 3958, 1973, 1720, 416, 4928, 199, 1397, 1209, 3718, 2015, 1086, 4972, 1835, 1869, 5115, 944, 5169, 1600, 3306, 2331, 3724, 5208, 2145, 1902, 4361, 2329, 4392, 1870, 4838, 4523, 228, 599, 471, 838, 4688, 4464, 4671, 4550, 3995, 3133, 468, 3984, 248, 4118, 3012, 746, 3080, 2484, 1924, 4617, 5154, 3649, 3528, 4654, 2707, 733, 2201, 1918, 4995, 3262, 2323, 3493, 2589, 3127, 3783, 2948, 3151, 2351, 1652, 4157, 2561, 88, 4365, 2503, 2759, 4904, 4314, 4848, 3269, 1057, 3635, 2110, 683, 4872, 1261, 144, 4992, 2459, 315, 589, 4614, 3668, 3247, 3024, 4632, 1937, 485, 2268, 3641, 4373, 787, 4219, 925, 1075, 3898, 4619, 1566, 3849, 5192, 4921, 5109, 4039, 2509, 2985, 5058, 1705, 2807, 100, 3167, 747, 1049, 811, 4630, 2677, 5249, 4398, 109, 5043, 1173, 4473]

# Making Independent Train set and Test Set

In [ ]:
X_train_independent = X.iloc[train, :]
y_train_independent = y.iloc[train]

X_test_independent = X.iloc[test, :]
y_test_independent = y.iloc[test]

# CV-10 with various Classifier ##

In [ ]:
classifier1 = SVC (C=1.0, kernel='poly', degree=5, gamma= 0.1, probability=True)
classifier2 = SVC (C=1.0, kernel='linear', degree=5, gamma= 0.1, probability=True)                   
classifier3 = xgboost.XGBClassifier(n_estimators = 200, base_score = 0.5)
classifier4 = GaussianNB()                
classifier5 = LogisticRegression()
classifier6 = RandomForestClassifier(n_estimators=200)
classifier7 = GradientBoostingClassifier(n_estimators=200)
classifier8 = AdaBoostClassifier(n_estimators=200,learning_rate=1)

classifier_list = []
classifier_list.append(classifier1)
classifier_list.append(classifier2)
classifier_list.append(classifier3)
classifier_list.append(classifier4)
classifier_list.append(classifier5)
classifier_list.append(classifier6)
classifier_list.append(classifier7)
classifier_list.append(classifier8)

curve_fp_list = []
curve_tp_list = []
curve_precision_list = []
curve_recall_list = []

for classifier in classifier_list:
    
    print(classifier)

    skf = StratifiedKFold(n_splits=10)
    accuracies = []
    aucs       = []
    auprs      = []
    precisions = [] 
    recalls    = []
    f1s        = []
    mccs       = []
    tp_list    = [] 
    mean_fp    = np.linspace(0, 1, 100)
    y_test_avg = []
    predictions_avg = []

    train_accuracies = []
    train_aucs       = []
    train_auprs      = []
    train_precisions = [] 
    train_recalls    = []
    train_f1s        = []
    train_mccs       = []
    train_y_test_avg = []
    train_predictions_avg = []
    
    count = 0
    threshold = 0.5

    print("CV Results:\n")

    for train_index, test_index in skf.split(X_train_independent, y_train_independent):

        X_train   = X_train_independent.iloc[train_index, :]
        y_train   = y_train_independent.iloc[train_index]

        X_test    = X_train_independent.iloc[test_index, :]
        y_test    = y_train_independent.iloc[test_index]

        accuracy, auc, aupr, precision, recall, f1, mcc, fp, tp, y_test, predictions, train_accuracy, train_auc, train_aupr, train_precision, train_recall, train_f1, train_mcc, train_y_test, train_predictions = scores(classifier, X_train, X_test, y_train, y_test)

        tp_list.append(interp(mean_fp, fp, tp))
        tp_list[-1][0] = 0.0

        accuracies.append(accuracy)
        aucs.append(auc)
        auprs.append(aupr)
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
        mccs.append(mcc)
        y_test_avg.append(y_test)
        predictions_avg.append(predictions)
        
        train_accuracies.append(train_accuracy)
        train_aucs.append(train_auc)
        train_auprs.append(train_aupr)
        train_precisions.append(train_precision)
        train_recalls.append(train_recall)
        train_f1s.append(train_f1)
        train_mccs.append(train_mcc)
        train_y_test_avg.append(train_y_test)
        train_predictions_avg.append(train_predictions)
        
        count += 1
        print("CV -", count)

        final_y_test      = np.concatenate(y_test_avg)
        final_predictions = np.concatenate(predictions_avg)
        mean_tp     = np.mean(tp_list, axis=0)
        mean_tp[-1] = 1.0
        precision_pos, recall_pos, _ = precision_recall_curve(final_y_test, final_predictions)

    average_accuracy  = np.mean(accuracies)
    average_auc       = np.mean(aucs)
    average_aupr      = np.mean(auprs)
    average_precision = np.mean(precisions)
    average_recall    = np.mean(recalls)
    average_f1        = np.mean(f1s)
    average_mcc       = np.mean(mcc)

    print("Accuracy\t:", average_accuracy)
    print("AUC\t\t:", average_auc)
    print("AUPR\t\t:", average_aupr)
    print("Precision\t:", average_precision)
    print("Recall\t\t:", average_recall)
    print("F1\t\t:", average_f1)
    print("MCC\t\t:", average_mcc)
    
    print("\nTrain Results:\n")
    
    train_average_accuracy  = np.mean(train_accuracies)
    train_average_auc       = np.mean(train_aucs)
    train_average_aupr      = np.mean(train_auprs)
    train_average_precision = np.mean(train_precisions)
    train_average_recall    = np.mean(train_recalls)
    train_average_f1        = np.mean(train_f1s)
    train_average_mcc       = np.mean(train_mcc)

    print("Accuracy\t:", train_average_accuracy)
    print("AUC\t\t:", train_average_auc)
    print("AUPR\t\t:", train_average_aupr)
    print("Precision\t:", train_average_precision)
    print("Recall\t\t:", train_average_recall)
    print("F1\t\t:", train_average_f1)
    print("MCC\t\t:", train_average_mcc)
    
    
    label = 'Train ROC(AUC = %.4f)' % average_auc
    pyplot.plot(mean_fp, mean_tp, color='red', label = label, lw=1.5, alpha=1)
    pyplot.plot([0, 1], [0, 1], color='black', linestyle = "--", lw=1.5, alpha=1)
    pyplot.xlim([-0.05, 1.05])
    pyplot.ylim([-0.05, 1.05])
    pyplot.xlabel('FP')
    pyplot.ylabel('TP')
    pyplot.title('ROC Curve')
    pyplot.legend(loc="lower right")
    pyplot.show()

    label = '(Train AUPR = %.4f)' % average_aupr
    pyplot.plot(recall_pos, precision_pos, color='green', label = label, lw=1.5, alpha=1)
    pyplot.plot([0, 1], [1, 0], color='black', linestyle = "--", lw=1.5, alpha=1)
    pyplot.xlim([-0.05, 1.05])
    pyplot.ylim([-0.05, 1.05])
    pyplot.xlabel('Recall')
    pyplot.ylabel('Precision')
    pyplot.title('AUPR Curve')
    pyplot.legend(loc="lower left")
    pyplot.show()


    print("\nIndependent Results:\n")
    accuracy, auc, aupr, precision, recall, f1, mcc, fp, tp, y_test, predictions = independent_test_scores(classifier, X_train_independent, X_test_independent, y_train_independent, y_test_independent)
    precision_pos, recall_pos, _ = precision_recall_curve(y_test, predictions)

    curve_fp_list.append(fp)
    curve_tp_list.append(tp)
    curve_precision_list.append(precision_pos)
    curve_recall_list.append(recall_pos)
    
    print("Accuracy\t:", accuracy)
    print("AUC\t\t:", auc)
    print("AUPR\t\t:", aupr)
    print("Precision\t:", precision)
    print("Recall\t\t:", recall)
    print("F1\t\t:", f1)
    print("MCC\t\t:", mcc)

    

colors = ['red', '#9c27b0', '#03a9f4', '#4caf50', 'orange', '#009688', 'yellow', '#ff1493']    
labels = ['SVC (Poly)', 'SVC (Linear)', 'XGBoost', 'GNB', 'LR', 'RF', 'GBoost', 'AdaBoost']

for i in range(len(curve_fp_list)):
    pyplot.plot(curve_fp_list[i], curve_tp_list[i], color=colors[i], label = labels[i], lw=1.5, alpha=1)
pyplot.plot([0, 1], [0, 1], color='black', linestyle = "--", lw=1.5, alpha=1)
pyplot.xlim([-0.05, 1.05])
pyplot.ylim([-0.05, 1.05])
pyplot.xlabel('False Positive')
pyplot.ylabel('True Positive')
pyplot.title('ROC Curve')
pyplot.legend(loc="lower right")
pyplot.show()

for i in range(len(curve_precision_list)):
    pyplot.plot(curve_recall_list[i], curve_precision_list[i], color=colors[i], label = labels[i], lw=1.5, alpha=1)
pyplot.plot([0, 1], [1, 0], color='black', linestyle = "--", lw=1.5, alpha=1)
pyplot.xlim([-0.05, 1.05])
pyplot.ylim([-0.05, 1.05])
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
pyplot.title('AUPR Curve')
pyplot.legend(loc="lower left")
pyplot.show()